In [88]:
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
import pickle
print ('Ran')

Ran


In [89]:
app_train = pd.read_csv('D:/Project_Data/application_train.csv')
app_test = pd.read_csv('D:/Project_Data/application_test.csv')

print("Shape of training data: {}".format(app_train.shape))
print("Shape of test data: {}".format(app_test.shape))

Shape of training data: (307511, 122)
Shape of test data: (48744, 121)


In [90]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
train_labels = app_train['TARGET']
train_sk_id_curr = app_train['SK_ID_CURR']
test_sk_id_curr = app_test['SK_ID_CURR']

app_train.drop('SK_ID_CURR', inplace=True, axis=1)
app_test.drop('SK_ID_CURR', inplace=True, axis=1)

app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 120)
Testing Features shape:  (48744, 120)


In [92]:
cat_features = [f for f in app_train.columns if app_train[f].dtype == 'object']

In [93]:
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, query_cols, sorter=sidx)]

In [94]:
cat_features_inds = column_index(app_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_inds)

Cat features are: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
[ 0  1  2  3  9 10 11 12 13 26 30 38 84 85 87 88]


In [95]:
for col in cat_features:
    app_train[col] = le.fit_transform(app_train[col].astype(str))
    app_test[col] = le.fit_transform(app_test[col].astype(str))

In [96]:
app_train
app_test

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,0,0,1,0,135000.0,568800.0,20560.5,450000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,1,0,99000.0,222768.0,17370.0,180000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,0,1,1,1,0,202500.0,663264.0,69777.0,630000.0,7,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,0,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,0,1,1,0,1,180000.0,625500.0,32067.0,625500.0,6,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,0,0,1,1,0,270000.0,959688.0,34600.5,810000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0
6,0,1,1,1,2,180000.0,499221.0,22117.5,373500.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
7,0,1,0,1,0,166500.0,180000.0,14220.0,180000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
8,0,0,0,1,0,315000.0,364896.0,28957.5,315000.0,6,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0
9,0,0,1,1,1,162000.0,45000.0,5337.0,45000.0,1,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [97]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(app_train)
app_train = imputer.transform(app_train)
app_test = imputer.transform(app_test)

In [98]:
app_train
app_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 3.],
       [0., 1., 1., ..., 0., 1., 4.],
       ...,
       [0., 0., 1., ..., 0., 3., 1.],
       [0., 1., 0., ..., 0., 0., 2.],
       [0., 0., 1., ..., 0., 1., 4.]])

In [99]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score

In [100]:
X_train, X_test, y_train, y_test = train_test_split(app_train, train_labels, test_size=0.2, stratify=train_labels, random_state=1)
print("Postive examples in train set: {}".format(np.sum(y_train==0)))
print("Negative examples in train set: {}".format(np.sum(y_train==1)))

print("Postive examples in test set: {}".format(np.sum(y_test==0)))
print("Negative examples in test set: {}".format(np.sum(y_test==1)))

Postive examples in train set: 226148
Negative examples in train set: 19860
Postive examples in test set: 56538
Negative examples in test set: 4965


In [101]:
clf = CatBoostClassifier(iterations=1000,
                              learning_rate=0.1,
                              depth=7,
                              l2_leaf_reg=40,
                              bootstrap_type='Bernoulli',
                              subsample=0.7,
                              scale_pos_weight=5,
                              eval_metric='AUC',
                              metric_period=50,
                              od_type='Iter',
                              od_wait=45,
                              random_seed=17,
                              allow_writing_files=False)

In [102]:
clf.fit(X_train, y_train,
             eval_set=(X_test, y_test),
             cat_features=cat_features_inds,
             use_best_model=True,
             verbose=True)

0:	test: 0.7013611	best: 0.7013611 (0)	total: 567ms	remaining: 9m 26s
50:	test: 0.7537932	best: 0.7537990 (49)	total: 24.6s	remaining: 7m 37s
100:	test: 0.7578963	best: 0.7578963 (100)	total: 47.2s	remaining: 6m 59s
150:	test: 0.7608482	best: 0.7608482 (150)	total: 1m 9s	remaining: 6m 30s
200:	test: 0.7627702	best: 0.7627702 (200)	total: 1m 33s	remaining: 6m 11s
250:	test: 0.7630244	best: 0.7631685 (246)	total: 1m 57s	remaining: 5m 49s
300:	test: 0.7635314	best: 0.7635314 (300)	total: 2m 20s	remaining: 5m 26s
350:	test: 0.7641911	best: 0.7643622 (341)	total: 2m 44s	remaining: 5m 4s
Stopped by overfitting detector  (45 iterations wait)

bestTest = 0.7643622304
bestIteration = 341

Shrink model to first 342 iterations.


In [103]:
from sklearn.metrics import accuracy_score
print('AUC:', roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]))
y_pred = clf.predict(X_test)
print('Accuracy score ',accuracy_score(y_test, y_pred))

AUC: 0.7643622304021603
Accuracy score  0.8708843471050193


In [104]:
predictions = clf.predict(app_test)

In [105]:
submission = pd.DataFrame({'SK_ID_CURR': test_sk_id_curr, 'TARGET': predictions})

In [106]:
submission.head()

,SK_ID_CURR,TARGET
0,100001,0.0
1,100005,0.0
2,100013,0.0
3,100028,0.0
4,100038,0.0


In [108]:
submission.to_csv('baseline_catboost_imputed_median.csv', index = False)